
# STEPS

1. Request data from API
2. Process the data
3. Save data to RDS

In [2]:
import requests
from pyjstat import pyjstat
import pandas as pd
import numpy as np
from collections import OrderedDict


# ENERGY CONSUMPTION

In [3]:
url = 'https://ec.europa.eu/eurostat/api/dissemination/statistics/1.0/data/TEN00123?format=JSON'
response = requests.get(url)
dataset = pyjstat.Dataset.read(response.text)
dataset

Dataset([('version', '2.0'),
         ('class', 'dataset'),
         ('label', 'Final energy consumption by product'),
         ('source', 'ESTAT'),
         ('updated', '2023-04-28T11:00:00+0200'),
         ('value',
          OrderedDict([('1008', 110.9),
                       ('1009', 138.974),
                       ('1010', 77.225),
                       ('1011', 67.005),
                       ('1012', 85.149),
                       ('1013', 94.089),
                       ('1014', 49.941),
                       ('1015', 123.149),
                       ('1016', 239.095),
                       ('1017', 123.502),
                       ('1018', 150.666),
                       ('1019', 227.071),
                       ('804', 364.679),
                       ('805', 340.942),
                       ('806', 344.577),
                       ('807', 345.239),
                       ('808', 340.804),
                       ('809', 352.912),
                       ('810', 360.998)

In [4]:
results = pyjstat.from_json_stat(response.json(object_pairs_hook=OrderedDict))
df =results[0]
df = df.rename(columns={'Geopolitical entity (reporting)': 'Country'})
df

,Time frequency,Energy balance,Standard international energy product classification (SIEC),Unit of measure,Country,Time,value
0,Annual,Final consumption - energy use,Total,Thousand tonnes of oil equivalent,European Union - 27 countries (from 2020),2010,973238.431
1,Annual,Final consumption - energy use,Total,Thousand tonnes of oil equivalent,European Union - 27 countries (from 2020),2011,933799.891
2,Annual,Final consumption - energy use,Total,Thousand tonnes of oil equivalent,European Union - 27 countries (from 2020),2012,933168.989
3,Annual,Final consumption - energy use,Total,Thousand tonnes of oil equivalent,European Union - 27 countries (from 2020),2013,931391.979
4,Annual,Final consumption - energy use,Total,Thousand tonnes of oil equivalent,European Union - 27 countries (from 2020),2014,890023.879
...,...,...,...,...,...,...,...
6475,Annual,Final consumption - energy use,Nuclear heat,Thousand tonnes of oil equivalent,Georgia,2017,NaN
6476,Annual,Final consumption - energy use,Nuclear heat,Thousand tonnes of oil equivalent,Georgia,2018,NaN
6477,Annual,Final consumption - energy use,Nuclear heat,Thousand tonnes of oil equivalent,Georgia,2019,NaN
6478,Annual,Final consumption - energy use,Nuclear heat,Thousand tonnes of oil equivalent,Georgia,2020,NaN


In [5]:
df_filtered = df.loc[df['Standard international energy product classification (SIEC)'] == 'Total']
df_filtered = df_filtered.drop(['Time frequency','Energy balance','Unit of measure','Standard international energy product classification (SIEC)'], axis =1)

In [6]:

nrg_con_total = df_filtered.pivot(index='Country' , columns='Time', values ='value')
nrg_con_total = nrg_con_total[~nrg_con_total.index.str.startswith('Euro')]
nrg_con_total = nrg_con_total.fillna(0)
nrg_con_total.head()

Time,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
Country,,,,,,,,,,,,
Albania,1897.918,1953.468,1790.623,1961.871,2058.574,1962.498,1899.558,2078.928,2134.212,2053.036,1840.668,2012.504
Austria,25961.403,25096.146,25203.967,25800.573,24762.326,25476.389,26099.940,26503.187,26042.688,26222.168,24873.297,26396.623
Belgium,35454.681,32624.012,33058.280,34107.252,31594.248,33155.375,33436.999,32979.018,33204.310,32589.709,30938.054,33181.638
Bosnia and Herzegovina,0.000,0.000,0.000,0.000,3103.624,3316.907,3543.934,3497.933,4211.931,4207.688,4024.489,4305.320
Bulgaria,8699.392,9125.105,9103.369,8680.757,8881.501,9388.553,9517.949,9745.903,9757.670,9719.600,9499.659,10140.235


In [33]:
for i, row in nrg_con_total.iterrows():
    print(i, row['2010'], row['2011'], row['2012'])

Albania 1897.918 1953.468 1790.623
Austria 25961.403 25096.146 25203.967
Belgium 35454.681 32624.012 33058.28
Bosnia and Herzegovina 0.0 0.0 0.0
Bulgaria 8699.392 9125.105 9103.369
Croatia 7129.817 6875.498 6558.631
Cyprus 1645.788 1614.212 1492.799
Czechia 24121.144 23337.059 23326.229
Denmark 14849.916 14116.296 13611.792
Estonia 2878.489 2781.689 2827.225
Finland 25043.343 23726.736 23952.27
France 146258.394 140774.818 145831.896
Georgia 0.0 0.0 0.0
Germany (until 1990 former territory of the FRG) 209923.09 199292.537 202662.536
Greece 18373.799 18185.332 16402.985
Hungary 16879.616 16917.517 16003.531
Iceland 2517.495 2521.164 2600.49
Ireland 11187.012 10361.568 10174.773
Italy 123052.729 117518.026 116559.044
Kosovo (under United Nations Security Council Resolution 1244/99) 1190.842 1266.098 1219.372
Latvia 4002.237 3751.161 3908.282
Liechtenstein 0.0 0.0 0.0
Lithuania 4757.259 4728.244 4838.968
Luxembourg 3895.179 3889.18 3799.238
Malta 401.35 388.461 407.257
Moldova 2268.192 23

# POPULATION

In [8]:
url = 'https://ec.europa.eu/eurostat/api/dissemination/statistics/1.0/data/DEMO_PJANGROUP?format=JSON'
population = requests.get(url)
#dataset = pyjstat.Dataset.read(response.text)
results1 = pyjstat.from_json_stat(population.json(object_pairs_hook=OrderedDict))
df1 =results1[0]
df1.head()

,Time frequency,Unit of measure,Sex,Age class,Geopolitical entity (reporting),Time,value
0,Annual,Number,Total,Total,European Union - 27 countries (from 2020),1960,354531754.0
1,Annual,Number,Total,Total,European Union - 27 countries (from 2020),1961,NaN
2,Annual,Number,Total,Total,European Union - 27 countries (from 2020),1962,NaN
3,Annual,Number,Total,Total,European Union - 27 countries (from 2020),1963,NaN
4,Annual,Number,Total,Total,European Union - 27 countries (from 2020),1964,NaN


In [9]:
df1 = df1.rename(columns={'Geopolitical entity (reporting)': 'Country'})
df1_filtered = df1.loc[df1['Sex'] == 'Total']
df1_filtered = df1_filtered.drop(['Time frequency','Unit of measure','Sex','Age class'], axis =1)
df1_filtered = df1_filtered[(df1_filtered['Time'] >= '2010') & (df1_filtered['Time'] <= '2021')]
df1_filtered

,Country,Time,value
50,European Union - 27 countries (from 2020),2010,440660421.0
51,European Union - 27 countries (from 2020),2011,439942305.0
52,European Union - 27 countries (from 2020),2012,440552661.0
53,European Union - 27 countries (from 2020),2013,441257711.0
54,European Union - 27 countries (from 2020),2014,442883888.0
...,...,...,...
84540,Azerbaijan,2017,0.0
84541,Azerbaijan,2018,0.0
84542,Azerbaijan,2019,0.0
84543,Azerbaijan,2020,0.0


In [10]:
pop_total = df1_filtered.pivot_table(index = 'Country', columns ='Time',values = 'value')
pop_total = pop_total[~pop_total.index.str.startswith('Euro')]
pop_total = pop_total.fillna(0)
pop_total.head()


Time,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
Country,,,,,,,,,,,,
Albania,272722.909091,2.722954e+05,2.721640e+05,271970.454545,271835.772727,271697.454545,271073.409091,271623.727273,271486.954545,271163.318182,270083.136364,268803.727273
Andorra,8053.272727,7.466091e+03,7.466091e+03,7285.636364,0.000000,0.000000,0.000000,0.000000,0.000000,7239.045455,0.000000,0.000000
Armenia,303780.000000,3.262650e+06,3.274285e+06,0.000000,0.000000,284241.045455,283511.136364,282549.954545,281342.318182,280546.136364,279862.318182,279865.727273
Austria,807794.045455,8.105665e+05,8.141785e+05,818587.136364,824298.045455,833266.272727,845478.409091,853677.772727,859029.545455,863416.181818,869443.954545,873252.227273
Azerbaijan,830976.136364,8.421882e+05,8.542381e+05,866076.863636,877891.000000,889086.181818,900123.409091,910067.227273,918261.000000,926004.863636,934000.454545,938497.681818
